In [33]:
import pandas as pd
import numpy as np
import xlsxwriter
import csv

In [34]:
# Import Excel files containing stand_init and tree_init tables
standinit = pd.read_excel("data/GNN_Deschutes_FVS_working.xlsx", sheetname = "FVS_StandInit")
treeinit = pd.read_excel("data/GNN_Deschutes_FVS_working.xlsx", sheetname = "FVS_TreeInit")

In [52]:
# Years in which to perform treatment (options are "none", "first", "second", and "both")
treatmentSchedule = "both"

# Climate projections we can use in our FVS simulations
climateProjections = ["none",
                "CCSM4_rcp45", "CCSM4_rcp60", "CCSM4_rcp85",
                "Ensemble_rcp45", "Ensemble_rcp60", "Ensemble_rcp85",
                "GFDLCM3_rcp45","GFDLCM3_rcp60","GFDLCM3_rcp85",
                "HadGEM2ES_rcp45","HadGEM2ES_rcp60","HadGEM2ES_rcp85"]
# Plant association groups to include in FVS simulations
forestedPAGs = ["LPD","MCW","MCD","MHD"]

# Generate the keyword file
with open("fvsDeschutesClimateChange_" + treatmentSchedule + ".key", "w") as keyfile:
    # Run separate site simulation for each climate projection
    for projection in [0,4,6]:
        # Iterate over stands
        for index, row in standinit.iterrows():
            # Use small subset of stands for testing purposes
            if index > 4:
                break
            # Only run FVS simulations for forested stands
            pag = row["Stand_CN"][len(row["Stand_CN"]) - 3:len(row["Stand_CN"])]
            if pag in forestedPAGs:
                writeColumnRuler(keyfile)
                writePreamble(keyfile, row)
                writeClimateKeys(keyfile, row, climateProjections[projection])
                writeInputDatabaseKeys(keyfile, row)
                writeSimCycleKeys(keyfile, row)
                writePotFireStatement(keyfile)
                defineOwlSpeciesGroup(keyfile)
                computeFVSVariables(keyfile)
                writeFFEKeywords(keyfile)
                if treatmentSchedule == "both":
                    writeTreatmentScheduling_both(keyfile)
                elif treatmentSchedule == "second":
                    writeTreatmentScheduling_second(keyfile)
                elif treatmentSchedule == "first":
                    writeTreatmentScheduling_first(keyfile)
                else:
                    writeTreatmentScheduling_none(keyfile)
                writeOutputDatabaseKeys(keyfile, treatmentSchedule)
                writeSiteSDICodeLabel(keyfile, pag)
                writeEndOfStand(keyfile)
    keyfile.write("Stop\n")
keyfile.closed

True

##Method definition cells

In [44]:
# Since the keyword files are column-oriented, make header to improve readability
def writeColumnRuler(outfile):
    outfile.write("*                                 Column ruler                                  \n")
    outfile.write("*---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8\n")
    outfile.write("*Keyword     Field 1   Field 2   Field 3   Field 4   Field 5   Field 6   Field 7\n\n")

In [46]:
# Identify the stand in the keyword file, instruct output to be written to the screen
def writePreamble(outfile, standrow):
    outfile.write("StdIdent\n")
    outfile.write(standrow["Stand_CN"] + "\n")
    outfile.write("StandCN\n" + standrow["Stand_CN"] + "\n\n")
    outfile.write("Screen\n\n")

In [19]:
# Keywords to produce the simulations under climate change
def writeClimateKeys(outfile, standrow, climProj):
    if climProj != "none":
        outfile.write("MGMTID\n")
        outfile.write(climProj + "\n")
        outfile.write("Climate\n")
        outfile.write("ClimData\n")
        outfile.write(climProj + "\n")
        outfile.write("../data/FVSClimAttrs_PAG.csv\n")
        outfile.write("ClimRept\n")
        outfile.write("End\n\n")

In [20]:
# Inform FVS where to find the stand and tree tables
def writeInputDatabaseKeys(outfile, standrow):
    outfile.write("Database\n")
    outfile.write("DSNIN\n")
    outfile.write("../data/GNN_Deschutes_FVS_working.xlsx\n")
    outfile.write("StandSQL\n")
    outfile.write("SELECT *\n")
    outfile.write("FROM [FVS_StandInit$]\n")
    outfile.write("WHERE Stand_CN = '" + standrow["Stand_CN"] + "'\n")
    outfile.write("EndSQL\n")
    outfile.write("TreeSQL\n")
    outfile.write("SELECT *\n")
    outfile.write("FROM [FVS_TreeInit$]\n")
    outfile.write("WHERE Stand_CN = " + standrow["Stand_CN"][standrow["Stand_CN"].find('_') + 1:standrow["Stand_CN"].rfind('_')] + "\n")
    outfile.write("EndSQL\n")
    outfile.write("End\n\n")

In [45]:
# Fix the "breakpoint" years in the simulations
def writeSimCycleKeys(outfile, standrow):
    # FVS automatically determines habitat type from database input, so skipping manual assignment
    #outfile.write("STDINFO                    450\n")
    # Determining number of cycles:
        # get to 2015 (variable)
        # simulate to 2025 when first treatment will occur (9 + 1)
        # simulate to 2035 when measurements to determine treatment in following period will occur (10)
        # simulate to 2045 when second treatment will occur (9 + 1)
        # simulate to end of planning horizon: 2095 (10 + 10 + 10 + 10 + 10 + 1)
    # Cycles of length 10 are default so no need to specify their length,
    # only specify those with non-decadal lengths
    outfile.write("InvYear        " + str(standrow["Inv_Year"]) + ".\n")
    outfile.write("NumCycle          12\n")
    outfile.write("TIMEINT           1.         " + str(2015 - standrow["Inv_Year"]) + "\n")
    outfile.write("TIMEINT           2.         9\n")
    outfile.write("TIMEINT           3.         1\n")
    outfile.write("TIMEINT           5.         9\n")
    outfile.write("TIMEINT           6.         1\n")
    outfile.write("TIMEINT          12.         1\n\n")

In [22]:
# Request potential fire report
def writePotFireStatement(outfile):
    outfile.write("FMIn\n")
    outfile.write("PotFire            0      200.        1.\n")
    outfile.write("End\n\n")

In [23]:
# Define which species are candidates for providing Northern Spotted Owl habitat
def defineOwlSpeciesGroup(outfile):
    outfile.write("SPGROUP      OWL\n")
    outfile.write("1 SP DF WF MH IC ES SH PP 11 GF AF SF NF WB RC WH OS WL PY\n\n")

In [24]:
# Compute custom variable values
def computeFVSVariables(outfile):
    outfile.write("Compute            0\n")
    outfile.write("CC_WJ = SPMCDBH(7, 11, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_WP = SPMCDBH(7, WP, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_SP = SPMCDBH(7, SP, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_DF = SPMCDBH(7, DF, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_WF = SPMCDBH(7, WF, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_MH = SPMCDBH(7, MH, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_IC = SPMCDBH(7, IC, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_LP = SPMCDBH(7, LP, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_ES = SPMCDBH(7, ES, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_SH = SPMCDBH(7, SH, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_PP = SPMCDBH(7, PP, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_AF = SPMCDBH(7, AF, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_SF = SPMCDBH(7, SF, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_NF = SPMCDBH(7, NF, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_WB = SPMCDBH(7, WB, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_WL = SPMCDBH(7, WL, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_WH = SPMCDBH(7, WH, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_AS = SPMCDBH(7,AS,0,0,999,0,999)\n")
    outfile.write("CC_CW = SPMCDBH(7, CW, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_CH = SPMCDBH(7, CH, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_WO = SPMCDBH(7, WO, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_MC = SPMCDBH(7, MC, 0, 0, 999, 0, 999)\n")
    outfile.write("CC_TOT = SPMCDBH(7, 0, 0, 0, 999, 0, 999)*0.586021495\n")
    outfile.write("QMD_TOT = SPMCDBH(5, 0, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_WJ = SPMCDBH(2, 11, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_WP = SPMCDBH(2, WP, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_SP = SPMCDBH(2, SP, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_DF = SPMCDBH(2, DF, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_WF = SPMCDBH(2, WF, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_MH18 = SPMCDBH(2, MH, 0, 18, 999, 0, 999)\n")
    outfile.write("BA_WF18 = SPMCDBH(2, WF, 0, 18, 999, 0, 999)\n")
    outfile.write("BA_MH = SPMCDBH(2, MH, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_IC = SPMCDBH(2, IC, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_LP = SPMCDBH(2, LP, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_ES = SPMCDBH(2, ES, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_SH = SPMCDBH(2, SH, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_PP = SPMCDBH(2, PP, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_OS = SPMCDBH(2, OS, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_GF = SPMCDBH(2, GF, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_AF = SPMCDBH(2, AF, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_SF = SPMCDBH(2, SF, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_NF = SPMCDBH(2, NF, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_WB = SPMCDBH(2, WB, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_WL = SPMCDBH(2, WL, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_WH = SPMCDBH(2, WH, 0, 0, 999, 0, 999)\n")
    outfile.write("BA_TOT = SPMCDBH(2, 0, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_WJ = SPMCDBH(1, 11, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_WP = SPMCDBH(1, WP, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_SP = SPMCDBH(1, SP, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_DF = SPMCDBH(1, DF, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_WF = SPMCDBH(1, WF, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_MH = SPMCDBH(1, MH, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_IC = SPMCDBH(1, IC, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_LP = SPMCDBH(1, LP, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_ES = SPMCDBH(1, ES, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_SH = SPMCDBH(1, SH, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_PP = SPMCDBH(1, PP, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_GF = SPMCDBH(1, GF, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_AF = SPMCDBH(1, AF, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_SF = SPMCDBH(1, SF, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_NF = SPMCDBH(1, NF, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_WB = SPMCDBH(1, WB, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_WL = SPMCDBH(1, WL, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_WH = SPMCDBH(1, WH, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_AS = SPMCDBH(1,AS,0,0,999,0,999)\n")
    outfile.write("TA_TOT = SPMCDBH(1, 0, 0, 0, 999, 0, 999)\n")
    outfile.write("TA_TOT7 = SPMCDBH(1, 0, 0, 0, 7, 0, 999)\n")
    outfile.write("TA_TOT12 = SPMCDBH(1, 0, 0, 0, 12, 0, 999)\n")
    outfile.write("TA_OWL = SPMCDBH(1, OWL, 0, 30, 999, 0, 999)\n")
    outfile.write("HG_TOT = SPMCDBH(1, 0, 0, 0, 999, 0, 999)\n")
    outfile.write("r6TALTOT = SPMCDBH(1, 0, 0, 20, 29.9, 0, 999)\n")
    outfile.write("SSTA = SPMCDBH(1, 0, 0, 0, 4.9, 0, 999)\n")
    outfile.write("PoleTA = SPMCDBH(1, 0, 0, 5, 8.9, 0, 999)\n")
    outfile.write("SmTA = SPMCDBH(1, 0, 0, 9, 20.9, 0, 999)\n")
    outfile.write("LgTA = SPMCDBH(1, 0, 0, 21, 999, 0, 999)\n")
    outfile.write("LrgsTA = MAXINDEX(SSTA, PoleTA, SmTA, LgTA)\n")
    outfile.write("SSCov = SPMCDBH(7, 0, 0, 0, 4.9, 0, 999)\n")
    outfile.write("PoleCov = SPMCDBH(7, 0, 0, 5, 8.9, 0, 999)\n")
    outfile.write("SmCov = SPMCDBH(7, 0, 0, 9, 20.9, 0, 999)\n")
    outfile.write("LgCov = SPMCDBH(7, 0, 0, 21, 999, 0, 999)\n")
    outfile.write("SDI = SPMCDBH(11, 0, 0, 0, 999, 0, 999)\n")
    outfile.write("SDI7 = SPMCDBH(11, 0, 0, 7, 999, 0, 999)\n")
    outfile.write("CRBULK = CrBulkDn\n")
    outfile.write("TRCInd = TORCHIDX\n")
    outfile.write("CRInd = CROWNIDX\n")
    outfile.write("SC_CODE = 0\n")
    outfile.write("SER_STG = 0\n")
    outfile.write("special = 0\n")
    outfile.write("viableCC = 0\n")
    outfile.write("FM1 = FuelMods(1, 1)\n")
    outfile.write("FM1_WGT = FuelMods(1, 2)\n")
    outfile.write("FM2 = FuelMods(2, 1)\n")
    outfile.write("FM2_WGT = FuelMods(2, 2)\n")
    outfile.write("tophght = ATopHt\n")
    outfile.write("CRHGT = CrBaseHt\n")
    outfile.write("snagGE10=Snags(1,All,0,10.,999.,0.,999.,0)\n")
    outfile.write("snagGE20=Snags(1,All,0,20.,999.,0.,999.,0)\n")
    outfile.write("fuel6_12 = FuelLoad(5, 5)\n")
    outfile.write("fuelGE12 = FuelLoad(6, 6)\n")
    outfile.write("fuel1h = FuelLoad(1, 1)\n")
    outfile.write("fuel10h = FuelLoad(2, 2)\n")
    outfile.write("fuel100h = FuelLoad(3, 3)\n")
    outfile.write("FLEN = POTFLEN(3)\n")
    outfile.write("FTP = POTFTYPE(1)\n")
    outfile.write("Dead21T = SpMcDBH(1,All,0,30.,200.,0.,500.,1,0.)\n")
    outfile.write("MORT_TPA = SpMcDBH(1,All,0,0,999,0.,999,1)\n")
    outfile.write("MORT_QMD = SpMcDBH(5,All,0,0,999,0.,999,1)\n")
    outfile.write("Live21T = SpMcDBH(1,All,0,30.,200.,0.,500.,0,0.)\n")
    outfile.write("pWFBA = (SPMCDBH(2, DF, 0, 6, 999, 0, 999, 3) + SPMCDBH(2, WF, 0, 6, 999, 0, 999, 3)) / ABA\n")
    outfile.write("pLPBA = SPMCDBH(2, LP, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pMHBA = SPMCDBH(2, MH, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pESBA = SPMCDBH(2, ES, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pPPBA = SPMCDBH(2, PP, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pWPBA = SPMCDBH(2, WP, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pSPBA = SPMCDBH(2, SP, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pSHBA = SPMCDBH(2, SH, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pAFBA = SPMCDBH(2, AF, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pSFBA = SPMCDBH(2, SF, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("pWBBA = SPMCDBH(2, WB, 0, 6, 999, 0, 999, 3) / ABA\n")
    outfile.write("r6Per = pWFBA + pLPBA + pMHBA + pESBA + pPPBA + pWPBA + pSPBA + pSHBA + pAFBA + pSFBA + pWBBA\n")
    outfile.write("r6PerM = max(0.001, r6Per)\n")
    outfile.write("fuelDuf = FuelLoad(11, 11)\n")
    outfile.write("fuelCW = FuelLoad(4, 9)\n")
    outfile.write("NUMSEED1 = LININT(RANN, 0, 1, 25, 75)\n")
    outfile.write("NUMSEED2 = LININT(RANN, 0, 1, 75, 125)\n")
    outfile.write("NUMSEED3 = LININT(RANN, 0, 1, 125, 275)\n")
    outfile.write("END\n\n")

In [25]:
# Keywords to control fuel pools' moisture and decay rates, species-specific decay rates,
# and instructs the FFE to use the new fuel model logic (53 fuel models) and the corresponding fire behavior
def writeFFEKeywords(outfile):
    outfile.write("Fmin\n")
    outfile.write("MOISTURE           0         3        4        8        10         3       75        39\n")
    outfile.write("FireCalc           0         1        2\n")
    outfile.write("FUELPOOL           1         1\n")
    outfile.write("FUELPOOL           2         1\n")
    outfile.write("FUELPOOL           3         1\n")
    outfile.write("FUELPOOL           4         3\n")
    outfile.write("FUELPOOL           5         2\n")
    outfile.write("FUELPOOL           6         1\n")
    outfile.write("FUELPOOL           7         2\n")
    outfile.write("FUELPOOL           8         2\n")
    outfile.write("FUELPOOL           9         3\n")
    outfile.write("FUELPOOL          10         3\n")
    outfile.write("FUELPOOL          11         1\n")
    outfile.write("FUELPOOL          12         3\n")
    outfile.write("FUELPOOL          13         3\n")
    outfile.write("FUELPOOL          14         3\n")
    outfile.write("FUELPOOL          15         3\n")
    outfile.write("FUELPOOL          16         1\n")
    outfile.write("FUELPOOL          17         1\n")
    outfile.write("FUELPOOL          18         1\n")
    outfile.write("FUELPOOL          19         2\n")
    outfile.write("FUELPOOL          20         1\n")
    outfile.write("FUELPOOL          21         4\n")
    outfile.write("FUELPOOL          22         4\n")
    outfile.write("FUELPOOL          23         4\n")
    outfile.write("FUELPOOL          24         4\n")
    outfile.write("FUELPOOL          25         4\n")
    outfile.write("FUELPOOL          26         4\n")
    outfile.write("FUELPOOL          27         3\n")
    outfile.write("FUELPOOL          28         4\n")
    outfile.write("FUELPOOL          29         3\n")
    outfile.write("FUELPOOL          30         4\n")
    outfile.write("FUELPOOL          31         4\n")
    outfile.write("FUELPOOL          32         1\n")
    outfile.write("FUELPOOL          33         4\n")
    outfile.write("Fueldcay           1       0.5     0.002     0.041     0.041     0.041     0.014\n")
    outfile.write("Fueldcay           2       0.5     0.002     0.043     0.043     0.043     0.017\n")
    outfile.write("Fueldcay           3       0.5     0.002     0.046     0.046     0.046     0.020\n")
    outfile.write("Fueldcay           4       0.5     0.002     0.055     0.055     0.055     0.031\n")
    outfile.write("END\n\n")

In [39]:
def writeTreatmentScheduling_both(outfile):
    outfile.write("*If statements to control treatment in both periods will go here\n")

In [40]:
def writeTreatmentScheduling_second(outfile):
    outfile.write("*If statements to control treatment in second period will go here\n")

In [41]:
def writeTreatmentScheduling_first(outfile):
    outfile.write("*If statements to control treatment in first period will go here\n")

In [42]:
def writeTreatmentScheduling_none(outfile):
    outfile.write("*No treatments, no statements\n")

In [47]:
# Inform FVS where to find the stand and tree tables
def writeOutputDatabaseKeys(outfile, treatmentSchedule):
    outfile.write("DataBase\n")
    outfile.write("DSNOut\n")
    outfile.write("../output/fvsDeschutesClimateChange_" + treatmentSchedule + "_out.mdb\n")
    outfile.write("Compute            1         0\n")
    outfile.write("END\n\n")

In [51]:
def writeSiteSDICodeLabel(outfile, pag):
    # ? ask if numbers 777, 55 and 85 change depending on PAG (or sth else)
    outfile.write("SDIMax           All       777                         55.00     85.00\n")
    # ? ask how this changes depending on PAG
    outfile.write("SiteCode          MH       48.        1.\n")
    outfile.write("SPLabel\n")
    outfile.write("All, " + pag + "\n\n")

In [49]:
# Write the closing line for the stand
def writeEndOfStand(outfile):
    outfile.write("Process\n\n")

In [ ]:
# Bunch of if statements...
*----------- Prescribed burn if SDI is low-------------------
*                                 Column ruler                                  
*---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8
*Keyword     Field 1   Field 2   Field 3   Field 4   Field 5   Field 6   Field 7
IF                20
SDI LT 35 AND ((YEAR GT 2020 AND YEAR LT 2030) &
OR (YEAR GT 2040 AND YEAR LT 2050))
Then
Fmin
SimFire            0     PARMS (3,2,65,1,60,4)
FlameAdj           0         0         0         0
End
Compute            0
TUC = 1
END
EndIf
Compute            0
TTA = 70
TUC = 1/2
END
IF                 0
SDI7 GT 35
Then
Compute            0
TTA = 0
END
EndIf
IF                 0
((YEAR GT 2012 AND YEAR LT 2021) OR (YEAR GT 2035 AND YEAR LT 2041))
Then
Compute            0
CBDen = CrBulkDn
FlMod = FuelMods(1, 1)
BTA = BA_TOT*0.055/CBDen
FRTP = POTFTYPE(1)
FLYT = 0
End
EndIf
*----- 'thinning + pile burn slash and fuels + prescribed burn --------
IF                 0
CBDen GT 0.037 AND TA_TOT7 GT 20 &
AND (FlMod GE 202 &
AND (BA_MH18 + BA_WF18) &
GT 80) AND (YEAR GT 2020 AND YEAR LT 2030) 
Then
Compute            0
FL = 1
TUC = 1
FLYT1 = 1
END
YARDLOSS           0         0         0         0
THINBTA            0     PARMS (TTA,1,0,7,0,999)
Fmin
PILEBURN           0     PARMS (1,100,10,100,0)
SimFire            0     PARMS (3,2,65,1,60,4)
FlameAdj           0         0         0         0
End
EndIf
IF                 0
(FlMod LT 161 AND FlMod NE 142) AND (YEAR GT 2040 AND YEAR LT 2050) AND FL EQ 1
Then
Fmin
SimFire            0     PARMS (3,2,65,1,60,4)
FlameAdj           0         0         0
End
Compute            0
TUC = 1
END
EndIf
IF                 0
CBDen GT 0.037 AND TA_TOT7 GT 20 &
AND ((FlMod GE 202) AND (BA_MH18 + BA_WF18) &
GT 80) AND (FLYT NE 1) AND (YEAR GT 2040 AND YEAR LT 2050) AND SDI7 GT 35
Then
YARDLOSS           0         0         0         0
THINBTA            0     PARMS (0,1,0,7,0,999)
Fmin
PILEBURN           0     PARMS (1,100,10,100,0)
End
Compute            0
TUC = 1
END
EndIf
*------- prescribed burn ----------
IF                 0
(NOT(CBDen GT 0.037 AND TA_TOT7 GT 20)) &
AND ((FlMod GE 202) &
AND (BA_MH18 + BA_WF18) &
GT 80) AND (YEAR GT 2020 AND YEAR LT 2030) 
Then
Compute            0
TUC = 1
FL1 = 2021
END
Fmin
SimFire            0     PARMS (3,2,65,1,60,4)
FlameAdj           0         0         0         0
End
EndIf
IF                 0
(FlMod LE 161 OR FlMod NE 142) AND (YEAR GT 2040 AND YEAR LT 2050) AND FL1 LT 2030
Then
Fmin
SimFire            0     PARMS (3,2,65,1,60,4)
FlameAdj           0         0         0         0
End
Compute            0
TUC = 1
FL1 = 0
END
EndIf
IF                 0
(NOT(CBDen GT 0.037 AND TA_TOT7 GT 20)) &
AND ((FlMod GE 202)  &
AND (BA_MH18 + BA_WF18) &
GT 80) AND (YEAR GT 2040 AND YEAR LT 2050) 
Then
Compute            0
TUC = 1
END
Fmin
SimFire            0     PARMS (3,2,65,1,60,4)
FlameAdj           0         0         0         0
End
EndIf
*------- thinning and slash and fuels pile&burn ----- 
*                                 Column ruler                                  
*---+----1----+----2----+----3----+----4----+----5----+----6----+----7----+----8
*Keyword     Field 1   Field 2   Field 3   Field 4   Field 5   Field 6   Field 7
IF                 0
(CBDen GT 0.037) AND TA_TOT7 GT 20 AND &
((FlMod GE 202) AND ((BA_MH18 + BA_WF18) & 
LT 80)) AND &
(YEAR GT 2020 AND YEAR LT 2030) 
Then
YARDLOSS           0         0         0         0
THINBTA           0.     PARMS (TTA,1,0,7,0,999)
Fmin
PILEBURN           0     PARMS (1,100,10,100,0)
End
FIXMORT            1     PARMS(0,0.05,7,999,0,0)
FIXMORT            1     PARMS(0,0.15,0,7,0,0)
Compute            0
TUC = 1
END
EndIf
IF                 0
(CBDen GT 0.037) AND TA_TOT7 GT 20 AND &
((FlMod GE 202) AND ((BA_MH18 + BA_WF18) &
LT 80)) AND (FLYT NE 1) AND &
(YEAR GT 2040 AND YEAR LT 2050) AND SDI7 GT 35
Then
YARDLOSS           0         0         0         0
THINBTA           0.     PARMS (0,1,0,7,0,999)
Fmin
PILEBURN           0     PARMS (1,100,10,100,0)
End
FIXMORT            1     PARMS(0,0.05,7,999,0,0)
FIXMORT            1     PARMS(0,0.15,0,7,0,0)
Compute            0
TUC = 1
END
EndIf
*------- thinning and slash pile&burn ----- 
IF                 0
CBDen GT 0.037 AND TA_TOT7 GT 20 AND &
(FlMod LT 161 AND FlMod NE 142 AND FLMod GT 6 OR &
((FRTP GT 1 OR CBDen GT 0.08) AND FlMod EQ 142)) AND (YEAR GE 2021 AND YEAR LT 2030) 
Then
YARDLOSS           0         0         0         0
THINBTA           0.     PARMS (TTA,1,0,7,0,999)
FIXMORT            1     PARMS(0,0.05,7,999,0,0)
FIXMORT            1     PARMS(0,0.15,0,7,0,0)
Compute            0
TUC = 1
FLYT = 1
END
EndIf
IF                 0
CBDen GT 0.037 AND TA_TOT7 GT 20 AND &
(FLYT NE 1) AND ((FlMod LT 161 AND FlMod NE 142 AND FLMod GT 6) &
OR ((FRTP GT 1 OR CBDen GT 0.08) AND FlMod EQ 142)) AND (YEAR GE 2041 AND YEAR LT 2050)
Then
YARDLOSS           0         0         0         0
THINBTA           0.     PARMS (0,1,0,7,0,999)
FIXMORT            1     PARMS(0,0.05,7,999,0,0)
FIXMORT            1     PARMS(0,0.15,0,7,0,0)
Compute            0
TUC = 1
END
EndIf
IF
(pWFBA + pLPBA) lt .10 AND ABA ge 150 AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95 &
 AND TUC eq 1
THEN
ESTAB
Natural            5    Parms(SH,NUMSEED1*pSHBA,100, 2.,0, 2)
Natural            5    Parms(WF,NUMSEED1*pWFBA,100, 2.,0, 1)
Natural            5    Parms(MH,NUMSEED1*pMHBA,100, 2.,0, 1)
Natural            5    Parms(LP,NUMSEED1*pLPBA,100, 2.,2, 0)
Natural            5    Parms(ES,NUMSEED1*pESBA,100, 2.,0, 0)
Natural            5    Parms(PP,NUMSEED1*pPPBA,100, 2.,0, 2)
Natural            5    Parms(AF,NUMSEED1*pAFBA,100, 2.,0, 1)
Natural            5    Parms(SF,NUMSEED1*pSFBA,100, 2.,0, 1)
Natural            5    Parms(SP,NUMSEED1*pSPBA,100, 2.,0, 0)
Natural            5    Parms(WP,NUMSEED1*pWPBA,100, 2.,0, 0)
Natural            5    Parms(WB,NUMSEED1*pWBBA,100, 2.,0, 0)
End
EndIf
IF
(pWFBA + pLPBA) lt .10 AND ABA ge 150 AND MOD(YEAR, 5) GT 0 AND &
CC_TOT LT 95 AND TUC eq 1/2
THEN
ESTAB
Natural            0    Parms(SH,NUMSEED1*pSHBA,100, 2.,0, 2)
Natural            0    Parms(WF,NUMSEED1*pWFBA,100, 2.,0, 1)
Natural            0    Parms(MH,NUMSEED1*pMHBA,100, 2.,0, 1)
Natural            0    Parms(LP,NUMSEED1*pLPBA,100, 2.,2, 0)
Natural            0    Parms(ES,NUMSEED1*pESBA,100, 2.,0, 0)
Natural            0    Parms(PP,NUMSEED1*pPPBA,100, 2.,0, 2)
Natural            0    Parms(AF,NUMSEED1*pAFBA,100, 2.,0, 1)
Natural            0    Parms(SF,NUMSEED1*pSFBA,100, 2.,0, 1)
Natural            0    Parms(SP,NUMSEED1*pSPBA,100, 2.,0, 0)
Natural            0    Parms(WP,NUMSEED1*pWPBA,100, 2.,0, 0)
Natural            0    Parms(WB,NUMSEED1*pWBBA,100, 2.,0, 0)
End
EndIf
IF
(pWFBA + pLPBA) lt .10 AND ABA lt 150 AND ABA ge 100 &
AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95 AND (TUC eq 1/2)
THEN
ESTAB
Natural            0    Parms(SH,NUMSEED2*pSHBA,100, 2.,0, 2)
Natural            0    Parms(WF,NUMSEED2*pWFBA,100, 2.,0, 1)
Natural            0    Parms(MH,NUMSEED2*pMHBA,100, 2.,0, 1)
Natural            0    Parms(LP,NUMSEED2*pLPBA,100, 2.,2, 0)
Natural            0    Parms(ES,NUMSEED2*pESBA,100, 2.,0, 0)
Natural            0    Parms(PP,NUMSEED2*pPPBA,100, 2.,0, 2)
Natural            0    Parms(AF,NUMSEED2*pAFBA,100, 2.,0, 1)
Natural            0    Parms(SF,NUMSEED2*pSFBA,100, 2.,0, 1)
Natural            0    Parms(SP,NUMSEED2*pSPBA,100, 2.,0, 0)
Natural            0    Parms(WP,NUMSEED2*pWPBA,100, 2.,0, 0)
Natural            0    Parms(WB,NUMSEED2*pWBBA,100, 2.,0, 0)
End
EndIf
IF
(pWFBA + pLPBA) lt .10 AND ABA lt 150 AND ABA ge 100 &
AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95
THEN
ESTAB
Natural            5    Parms(SH,(NUMSEED2*pSHBA)*TUC,100, 2.,0, 2)
Natural            5    Parms(WF,(NUMSEED2*pWFBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(MH,(NUMSEED2*pMHBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(LP,(NUMSEED2*pLPBA)*TUC,100, 2.,2, 0)
Natural            5    Parms(ES,(NUMSEED2*pESBA)*TUC,100, 2.,0, 0)
Natural            5    Parms(PP,(NUMSEED2*pPPBA)*TUC,100, 2.,0, 2)
Natural            5    Parms(AF,(NUMSEED2*pAFBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(SF,(NUMSEED2*pSFBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(SP,(NUMSEED2*pSPBA)*TUC,100, 2.,0, 0)
Natural            5    Parms(WP,(NUMSEED2*pWPBA)*TUC,100, 2.,0, 0)
Natural            5    Parms(WB,(NUMSEED2*pWBBA)*TUC,100, 2.,0, 0)
End
EndIf
IF
pWFBA + pLPBA lt .10 AND ABA lt 100 AND MOD(YEAR, 5) GT 0 &
AND CC_TOT LT 95 and (TUC eq 1/2)
THEN
ESTAB
Natural            0    Parms(SH,NUMSEED3*pSHBA,100, 2.,0, 2)
Natural            0    Parms(WF,NUMSEED3*pWFBA,100, 2.,0, 1)
Natural            0    Parms(MH,NUMSEED3*pMHBA,100, 2.,0, 1)
Natural            0    Parms(LP,NUMSEED3*pLPBA,100, 2.,2, 0)
Natural            0    Parms(ES,NUMSEED3*pESBA,100, 2.,0, 0)
Natural            0    Parms(PP,NUMSEED3*pPPBA,100, 2.,0, 2)
Natural            0    Parms(AF,NUMSEED3*pAFBA,100, 2.,0, 1)
Natural            0    Parms(SF,NUMSEED3*pSFBA,100, 2.,0, 1)
Natural            0    Parms(SP,NUMSEED3*pSPBA,100, 2.,0, 0)
Natural            0    Parms(WP,NUMSEED3*pWPBA,100, 2.,0, 0)
Natural            0    Parms(WB,NUMSEED3*pWBBA,100, 2.,0, 0)
End
EndIf
IF
pWFBA + pLPBA lt .10 AND ABA lt 100 AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95
THEN
ESTAB
Natural            5    Parms(SH,(NUMSEED3*pSHBA)*TUC,100, 2.,0, 2)
Natural            5    Parms(WF,(NUMSEED3*pWFBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(MH,(NUMSEED3*pMHBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(LP,(NUMSEED3*pLPBA)*TUC,100, 2.,2, 0)
Natural            5    Parms(ES,(NUMSEED3*pESBA)*TUC,100, 2.,0, 0)
Natural            5    Parms(PP,(NUMSEED3*pPPBA)*TUC,100, 2.,0, 2)
Natural            5    Parms(AF,(NUMSEED3*pAFBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(SF,(NUMSEED3*pSFBA)*TUC,100, 2.,0, 1)
Natural            5    Parms(SP,(NUMSEED3*pSPBA)*TUC,100, 2.,0, 0)
Natural            5    Parms(WP,(NUMSEED3*pWPBA)*TUC,100, 2.,0, 0)
Natural            5    Parms(WB,(NUMSEED3*pWBBA)*TUC,100, 2.,0, 0)
End
EndIf
IF
pWFBA + pLPBA ge .10 AND ABA ge 150 AND MOD(YEAR, 5) GT 0 &
AND CC_TOT LT 95 AND TUC EQ 1/2
THEN
ESTAB
Natural            0    Parms(WP, NUMSEED1*(.20*pWPBA/r6PerM),100, 2.,0, 0)

Natural            0    Parms(SP, NUMSEED1*(.20*pSPBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(WF, NUMSEED1*(.80*pWFBA/(pWFBA + pLPBA))&
,100,2.,0,1)
Natural            0    Parms(MH, NUMSEED1*(.20*pMHBA/r6PerM),100, 2.,0, 1)
Natural            0    Parms(LP, NUMSEED1*(.80*pLPBA/(pWFBA + pLPBA))&
,100,2.,2,0)
Natural            0    Parms(ES, NUMSEED1*(.20*pESBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(SH, NUMSEED1*(.20*pSHBA/r6PerM),100, 2.,0, 2)
Natural            0    Parms(PP, NUMSEED1*(.20*pPPBA/r6PerM),100, 2.,0, 2)
Natural            0    Parms(WB, NUMSEED1*(.20*pWBBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(AF, NUMSEED1*(.20*pAFBA/r6PerM),100, 2.,0, 1)
Natural            0    Parms(SF, NUMSEED1*(.20*pSFBA/r6PerM),100, 2.,0, 1)
End
EndIf
IF
pWFBA + pLPBA ge .10 AND ABA ge 150 AND MOD(YEAR, 5) GT 0 &
AND CC_TOT LT 95 AND TUC eq 1
THEN
ESTAB
Natural            5    Parms(WP, NUMSEED1*(.20*pWPBA/r6PerM),100, 2.,0, 0)

Natural            5    Parms(SP, NUMSEED1*(.20*pSPBA/r6PerM),100, 2.,0, 0)
Natural            5    Parms(WF, NUMSEED1*(.80*pWFBA/(pWFBA + pLPBA))&
,100,2.,0,1)
Natural            5    Parms(MH, NUMSEED1*(.20*pMHBA/r6PerM),100, 2.,0, 1)
Natural            5    Parms(LP, NUMSEED1*(.80*pLPBA/(pWFBA + pLPBA))&
,100,2.,2,0)
Natural            5    Parms(ES, NUMSEED1*(.20*pESBA/r6PerM),100, 2.,0, 0)
Natural            5    Parms(SH, NUMSEED1*(.20*pSHBA/r6PerM),100, 2.,0, 2)
Natural            5    Parms(PP, NUMSEED1*(.20*pPPBA/r6PerM),100, 2.,0, 2)
Natural            5    Parms(WB, NUMSEED1*(.20*pWBBA/r6PerM),100, 2.,0, 0)
Natural            5    Parms(AF, NUMSEED1*(.20*pAFBA/r6PerM),100, 2.,0, 1)
Natural            5    Parms(SF, NUMSEED1*(.20*pSFBA/r6PerM),100, 2.,0, 1)
End
EndIf
IF
(pWFBA + pLPBA) ge .10 AND ABA lt 150 AND ABA ge 100 & 
AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95 AND (TUC eq 1/2)
THEN
ESTAB
Natural            0    Parms(WP, NUMSEED2*(.20*pWPBA/r6PerM),100, 2.,0, 0)

Natural            0    Parms(SP, NUMSEED2*(.20*pSPBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(WF, NUMSEED2*(.80*pWFBA/(pWFBA + pLPBA))&
,100,2.,0,1)
Natural            0    Parms(MH, NUMSEED2*(.20*pMHBA/r6PerM),100, 2.,0, 1)
Natural            0    Parms(LP, NUMSEED2*(.80*pLPBA/(pWFBA + pLPBA))&
,100,2.,2,0)
Natural            0    Parms(ES, NUMSEED2*(.20*pESBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(SH, NUMSEED2*(.20*pSHBA/r6PerM),100, 2.,0, 2)
Natural            0    Parms(PP, NUMSEED2*(.20*pPPBA/r6PerM),100, 2.,0, 2)
Natural            0    Parms(WB, NUMSEED2*(.20*pWBBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(AF, NUMSEED2*(.20*pAFBA/r6PerM),100, 2.,0, 1)
Natural            0    Parms(SF, NUMSEED2*(.20*pSFBA/r6PerM),100, 2.,0, 1)
End
EndIf
IF
(pWFBA + pLPBA) ge .10 AND ABA lt 150 AND ABA ge 100 & 
AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95
THEN
ESTAB
Natural            5    Parms(WP, NUMSEED2*(.20*pWPBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(SP, NUMSEED2*(.20*pSPBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(WF, NUMSEED2*(.80*pWFBA/(pWFBA + pLPBA))*TUC&
,100,2.,0,0)
Natural            5    Parms(MH, NUMSEED2*(.20*pMHBA/r6PerM)*TUC,100, 2.,0, 1)
Natural            5    Parms(LP, NUMSEED2*(.80*pLPBA/(pWFBA + pLPBA))*TUC&
,100,2.,2,0)
Natural            5    Parms(ES, NUMSEED2*(.20*pESBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(SH, NUMSEED2*(.20*pSHBA/r6PerM)*TUC,100, 2.,0, 2)
Natural            5    Parms(PP, NUMSEED2*(.20*pPPBA/r6PerM)*TUC,100, 2.,0, 2)
Natural            5    Parms(WB, NUMSEED2*(.20*pWBBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(AF, NUMSEED2*(.20*pAFBA/r6PerM)*TUC,100, 2.,0, 1)
Natural            5    Parms(SF, NUMSEED2*(.20*pSFBA/r6PerM)*TUC,100, 2.,0, 1)
End
EndIf
IF
pWFBA + pLPBA ge .10 AND ABA lt 100 AND MOD(YEAR, 5) GT 0 &
AND CC_TOT LT 95 AND (TUC eq 1/2)
THEN
ESTAB
Natural            0    Parms(WP, NUMSEED3*(.20*pWPBA/r6PerM),100, 2.,0, 0)

Natural            0    Parms(SP, NUMSEED3*(.20*pSPBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(WF, NUMSEED3*(.80*pWFBA/(pWFBA + pLPBA))&
,100,2.,0,1)
Natural            0    Parms(MH, NUMSEED3*(.20*pMHBA/r6PerM),100, 2.,0, 1)
Natural            0    Parms(LP, NUMSEED3*(.80*pLPBA/(pWFBA + pLPBA))&
,100,2.,2,0)
Natural            0    Parms(ES, NUMSEED3*(.20*pESBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(SH, NUMSEED3*(.20*pSHBA/r6PerM),100, 2.,0, 2)
Natural            0    Parms(PP, NUMSEED3*(.20*pPPBA/r6PerM),100, 2.,0, 2)
Natural            0    Parms(WB, NUMSEED3*(.20*pWBBA/r6PerM),100, 2.,0, 0)
Natural            0    Parms(AF, NUMSEED3*(.20*pAFBA/r6PerM),100, 2.,0, 1)
Natural            0    Parms(SF, NUMSEED3*(.20*pSFBA/r6PerM),100, 2.,0, 1)
End
EndIf
IF
pWFBA + pLPBA ge .10 AND ABA lt 100 AND MOD(YEAR, 5) GT 0 AND CC_TOT LT 95
THEN
ESTAB
Natural            5    Parms(WP, NUMSEED3*(.20*pWPBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(SP, NUMSEED3*(.20*pSPBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(WF, NUMSEED3*(.80*pWFBA/(pWFBA + pLPBA))*TUC&
,100,2.,0,0)
Natural            5    Parms(MH, NUMSEED3*(.20*pMHBA/r6PerM)*TUC,100, 2.,0, 1)
Natural            5    Parms(LP, NUMSEED3*(.80*pLPBA/(pWFBA + pLPBA))*TUC&
,100,2.,2,0)
Natural            5    Parms(ES, NUMSEED3*(.20*pESBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(SH, NUMSEED3*(.20*pSHBA/r6PerM)*TUC,100, 2.,0, 2)
Natural            5    Parms(PP, NUMSEED3*(.20*pPPBA/r6PerM)*TUC,100, 2.,0, 2)
Natural            5    Parms(WB, NUMSEED3*(.20*pWBBA/r6PerM)*TUC,100, 2.,0, 0)
Natural            5    Parms(AF, NUMSEED3*(.20*pAFBA/r6PerM)*TUC,100, 2.,0, 1)
Natural            5    Parms(SF, NUMSEED3*(.20*pSFBA/r6PerM)*TUC,100, 2.,0, 1)
End
EndIf

##Experimental cells

In [15]:
forestedPAGs = ["LPD","MCW","MCD","MHD"]
for index,row in standinit.iterrows():
    pag = row["Stand_CN"][len(row["Stand_CN"]) - 3:len(row["Stand_CN"])]
    print str(row["Inv_Year"])
    if index > 4:
        break

2007
1996
2005
2011
1998
2010


##Archived cells

In [ ]:
# Needed to add quotes to the non-numeric fields in the ClimateAttrs file

#climcsv = read_csv("D:/Users/nkullman/Documents/Deschutes_Drink/ClimateFVS_returnedData/Stand_GNNLevel/Stand_FCID/FVSClimAttrs_PAG.csv")
#climcsv.to_csv("D:/Users/nkullman/Documents/Deschutes_Drink/ClimateFVS_returnedData/Stand_GNNLevel/Stand_FCID/FVSClimAttrs_PAG_py.csv",
              #index=None, quoting=csv.QUOTE_NONNUMERIC)